In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [6]:
print(train_data[0],'\n', train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 
 15.2


# Preparing the data

In [7]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

# Building network

In [8]:
from keras import models
from keras import layers    

In [15]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [18]:
num_epochs = 100
model = build_model()
history = model.fit(train_data, train_targets, epochs=num_epochs, batch_size=1, verbose=0)

In [19]:
history.history

{'loss': [170.8235012603114,
  23.9178315964991,
  18.49255056773638,
  15.464263814737789,
  14.284531325794447,
  12.434553716649061,
  12.010943956181523,
  11.996551729882395,
  11.434464413879079,
  10.722745206410908,
  10.372225574210514,
  10.626270476816034,
  10.219814110670717,
  10.021720532338696,
  10.02717601820247,
  9.37619771368756,
  9.2833580497922,
  9.473459479436558,
  9.11160145867207,
  8.960503373066281,
  8.653384624642841,
  8.903288706635832,
  8.478339256650866,
  8.699054471015998,
  8.278768229524676,
  8.159334754332987,
  8.221286901148241,
  8.151084053114186,
  7.830079010527049,
  7.890284537289022,
  7.725563180120232,
  7.630956945908471,
  7.928487504902765,
  7.577985601065506,
  7.333630393631093,
  7.275474307968967,
  6.982982550083276,
  7.401130197855723,
  7.035051044211619,
  6.601637412508227,
  6.874386879783914,
  6.577931231346841,
  6.735870583837714,
  6.737797510623885,
  6.750902633004624,
  6.228101502718121,
  6.314256364523173,

In [20]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [21]:
all_scores

[1.8006188798658918,
 2.3929606853145184,
 2.6379584026808787,
 2.5337753649985437]

In [22]:
test_mse, test_mae = model.evaluate(test_data, test_targets, verbose=0)

In [24]:
test_mae

3.0786366182215072